In [1]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["LUNARY_PUBLIC_KEY"] = "..."

load_dotenv(Path("../.env.test"), override=True)

True

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from openai import OpenAI
import lunary

openai = OpenAI()
lunary.monitor(openai)

In [4]:
def langchain_chain(inputs):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        ),
        ("user", "{question}"),
    ]

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(inputs)
    return answer.split("#### ")[-1]

In [5]:
evalset = lunary.get_dataset("gsm8k-evalset")

scores = []
for item in evalset:
    answer = langchain_chain(item.input)
    passed, results = lunary.evaluate(
        "exact-match",
        input=item.input,
        output=answer,
        ideal_output=item.ideal_output.split("#### ")[-1],
    )
    scores.append(int(passed))

print("Average score", sum(scores) / len(scores))

Average score 0.4


In [8]:
from zenbase.types import LMRequest, deflm


@deflm
def zen_chain(request: LMRequest):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        )
    ]

    for demo in request.zenbase.demos:
        messages += [
            ("user", demo.inputs),
            ("assistant", demo.outputs),
        ]

    messages.append(("user", "{question}"))

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(request.inputs)
    return answer.split("#### ")[-1]

In [9]:
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot
from zenbase.helpers.lunary import ZenLunary

demoset = ZenLunary.dataset_to_demos(lunary.get_dataset("gsm8k-demoset"))
optimizer = LabeledFewShot(demoset=demoset, shots=3)

best_fn, candidate_results = optimizer.perform(
    zen_chain,
    evaluator=ZenLunary.metric_evaluator(
        checklist="exact-match",
        evalset=evalset,
        concurrency=5,
    ),
    samples=4,
    rounds=1,
)

In [10]:
output = best_fn({"question": "What is 2+2?"})
output

'4'

In [11]:
# You can even run your function asynchronously in a coroutine
%autoawait

await best_fn.coroutine({
  "question": "What is 2+2?"
})

IPython autoawait is `on`, and set to use `asyncio`


'4'

In [12]:
# You can also save the zenbase params for re-use
import pickle

pickled_zenbase = pickle.dumps(best_fn.zenbase)
zen_chain.zenbase = pickle.loads(pickled_zenbase)

zen_chain({"question": "What is 2 + 2?"}) # uses the best few-shot demos

'4'